In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import load_model
from keras.metrics import get as get_metric
import numpy as np
import pandas as pd
from PIL import Image
from time import sleep, perf_counter
from sklearn.model_selection import train_test_split
from itertools import chain as flatten
from tkinter import Tk, Canvas, StringVar, Label, Button
from random import choice, randint
from math import dist, sqrt, pi

from os import getpid, environ
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import get_logger
get_logger().setLevel('ERROR')

from multiprocess import Pool
from multiprocess import Array, Process, cpu_count

imports = (Sequential, Dense, load_model, np, Image, train_test_split)  # no clue why globals() doesnt get passed to Pool()

train = pd.read_csv('train.csv')
y, test = np.asarray(train.pop('label')).astype('int32').reshape((-1,1)), pd.read_csv('test.csv')


def save_output(model, data, output='predictions.csv'):
    pd.DataFrame(columns=['ImageId', 'Label'], data=zip(range(1, len(data) + 1), map(lambda x: x.argmax(), model(data)))).to_csv(output, index=0)
    ! kaggle competitions submit -c digit-recognizer -f $output -m ""
    
    
main_params = {
    'model': 
    {
        'optimizer': 'RMSProp',
        'activation': 'sigmoid',
        'hidden_inc': (1, 1.25, 1),
        'output_size': 10,
        'hidden_size': 1.2,
        'loss_fn': 'sparse_categorical_crossentropy',
        'hidden_layers': 4,
        'metric': 'sparse_categorical_accuracy'
    },
     'fit':
    {
        'epochs': 100, 
        'batch_size': 105,
        'verbose': 0
    }
}

secondary_params = {
    'model': 
    {
        'optimizer': 'adam',
        'activation': 'sigmoid',
        'hidden_inc': (1, 1.25, 1),
        'output_size': 1,
        'hidden_size': 1.2,
        'loss_fn': 'binary_focal_crossentropy',
        'hidden_layers': 4,
        'metric': 'binary_accuracy'
    },
     'fit':
    {
        'epochs': 150, 
        'batch_size': 105,
        'verbose': 0
    }
}


class GridSearch:
    """
    all calculations treat the speed of the computer as a constant, although it should slow with higher values of s

    k := 11, since we have 11 cores to work with, since that'll be easiest to run
    C(s) ≈ 3.8, but slows down with more models being trained, so I'll make it a function of s
    F(args) = T(args) < kC(s) ? sC(s) + T(args) : sT(args)
    s := Π{0, L(args)} L(args_i)


    e = 10, b = 50 -> T(args) = 45.36

    M is the time of a batch of size b to be ran, L is the length of our dataset (31500 with train_test_split), C is an unknown constant added to the training time of loading the model, initializing training plus delays + printing
    T(args) := C + e * Σ{0, L/b} M(b_i) ≈ eLM(b)/b + C  # (since time shouldnt vary too much, especially for large e)
    T(args) ≈ eLM(b)/b + C

    using possibly flawed numbers from above and M(50) approximated to 7 ms (from basic_model's training), which actually varied from 7 to 9, mostly at 7
    45.36 ≈ C + (10)(31500)(0.007)(10)/(50) -> 44.1 + C = 45.36 -> C = 1.26

    T(args) ≈ 1.26 + eLM(b)/b)

    Worst case: s(1.26 + eLM(b)/b)
    Best case:  1.26 + 3.8s + eLM(b)/b)

    for 1620 models, was taking >= 32 hours, worst case was 28.92, e,b=20,[50,90] !!
    """
    model_options = {} 
    fit_options = {}
    
     
    @classmethod
    def load(cls, file='gridsearch.txt'):
        text, c, c2 = [x.split('|') for x in open(file).read().split('\n')], lambda s: tuple(map(lambda x: int(x) if float(x).is_integer() else float(x), s[1:-1].split(', '))), lambda s: tuple(map(str, s[1:-1].split(', ')))
        types = text[1]; l = {tuple([(c if t=='tint' else (c2 if t=='tstr' else globals()['__builtins__'].__dict__[t]))(a) for t, a in zip(types, x[:-1])]): float(x[-1]) for x in text[2:]}
        return cls(data=l, options={t: list(zip(*list(l.keys())))[i] for i, t in enumerate(text[0])}, load=True)
    
    def save(self, file='gridsearch.txt'):     
        titles, types, data = (['f_' + f for f in self.fit_options.keys()] + ['m_' + m for m in self.model_options], [(type(v).__name__ if type(v) != tuple else ('t' + ('int' if type(v[0]) == int else 'str'))) for v in list(self.results.keys())[0]], [(*a, b) for a, b in self.results.items()])
        open(file, 'w').write('\n'.join(('|'.join(titles), '|'.join(types), '\n'.join('|'.join(map(str, a)) for a in data))))
    
    def __init__(self, *args, load=False, **kwargs):
        self.imports = imports
        
        if load: self._from_load(*args, **kwargs)
        else: self.new(*args, **kwargs)
        self.options = self.fit_options | self.model_options
        
        (self._best_params, self.best), (self._worst_params, self.worst) = max(self.results.items(), key=lambda x: x[1]), min(self.results.items(), key=lambda x: x[1])
        self._best_params, self._worst_params = dict(zip(self.options.keys(), self._best_params)), dict(zip(self.options.keys(), self._worst_params))
        
    def new(self, model, X, y, parameters):
        self._start = perf_counter()
        
        def transform(dictionary):
            return {k: [v] if type(v) != list else v for k, v in dictionary.items()}
        
        self.model_options, self.fit_options = transform(parameters['model']), transform(parameters['fit'])
        self.model_combinations, self.fit_combinations = self.get_combinations(self.model_options), self.get_combinations(self.fit_options)
        
        self.total_combinations = len(self.model_combinations) * len(self.fit_combinations)
        print(f'training {self.total_combinations} models')

        self.model, self.X, self.y = model, X, y
        
        self.results = dict(sorted(self.search().items(), key=lambda x: x[1]))
        print(f'total time: {perf_counter() - self._start:.3f} seconds')
    
    def _from_load(self, options, data):
        self.options = options
        
        self.fit_options = {k[2:]: v for k, v in self.options.items() if k[:2] == 'f_'}
        self.model_options = {k[2:]: v for k, v in self.options.items() if k[:2] == 'm_'}
        
        self.results = data
        
        split = len(self.fit_options)
        self.fit_combinations, self.model_combinations = list(zip(*[(k[:split], k[split:]) for k in data.keys()]))
        self.total_combinations = len(data)
        
    @staticmethod
    def get_combinations(options):
        c = [(a,) for a in list(options.values())[0]]
                
        for values in list(options.values())[1:]:
            c = flatten.from_iterable([[(*part, a) for a in values] for part in c])
        return [dict(zip(options.keys(), a)) for a in c]

    def search2(self):
        def set_up():
            from psutil import Process as p, BELOW_NORMAL_PRIORITY_CLASS as b
            from os import getpid as g, environ as e
            p(g()).nice(b)
            e['TF_CPP_MIN_LOG_LEVEL'] = '3'
            
        return (lambda c: {tuple((f|m).values()) for (m,f),y in zip(c,Pool(cpu_count() - 1, set_up).starmap(lambda m,f: self.model(self.X, self.y, imports=self.imports, **m).fit(**f, verbose=0).score,c))})(list(flatten.from_iterable([[(a, b) for b in self.fit_combinations] for a in self.model_combinations])))
    
    def search(self):
        def get_score(i, s, m, f):
            from psutil import Process as p, BELOW_NORMAL_PRIORITY_CLASS as b
            from os import getpid as g, environ as e
            from time import perf_counter as t
            p(g()).nice(b)
            e['TF_CPP_MIN_LOG_LEVEL'] = '3'

            print(f'[{t() - self._start:.3f}] model #{i} starting training')
            s[i] = self.model(self.X, self.y, imports=self.imports, **m).fit(**f, verbose=0).score
            print(f"{f'[{t() - self._start:.3f}] model #{i} finished training':>75}")

        def progress_bar(a):
            return f'\r{a:>5} / {self.total_combinations} |{"="*(50*(a-len(processes))//self.total_combinations)+">":-<49}|'
            
        combs = list(flatten.from_iterable([(a, b) for b in self.fit_combinations] for a in self.model_combinations))  # list bc flatten returns a generator
        scores = Array('f', self.total_combinations)

        k, processes = cpu_count() - 1, []
        def clean():
            for p in processes:
                if not p.is_alive(): 
                    processes.remove(p)
                    p.join(); p.close()
                    
        for i, (a, b) in enumerate(combs):
            p = Process(target=get_score, args=(i, scores, a, b))
            processes.append(p)
            p.start()
            
            # print(progress_bar(i), end='')
            while len(processes) == k:  # TODO: redo this with one loop, where it looks at k - len(processes), gets that many combs, creates processes and then runs those
                clean(); sleep(0.1)
                

        while processes:
            clean()
            sleep(2)

        print('\nDone')
        return {tuple((fit | model).values()): score for (model, fit), score in zip(combs, scores)}

    def get_best_model(self, override_a={}, override_b={}):
        a, b = self.best_params
        a.update(override_a), b.update(override_b)
        return self.model(self.X, self.y, **a).fit(**b)
    
    @property
    def best_params(self): return {k: v for k, v in self._best_params.items() if k in self.model_options}, {k: v for k, v in self._best_params.items() if k in self.fit_options}

    @property
    def worst_params(self): return {k: v for k, v in self._worst_params.items() if k in self.model_options}, {k: v for k, v in self._worst_params.items() if k in self.fit_options}


def time_estimate(args, l=31500):
    mean = lambda x: sum(x) / len(x) if type(x) == list else x
    M = 0.007
    C = 3.8
    
    s = 1
    for options in args.values():
        for sublist in options.values():
            s *= len(sublist) if type(sublist) == list else 1
            
    e, b = mean(args['fit']['epochs']), mean(args['fit']['batch_size'])
    T = lambda e, b: 1.26 + M * e * l / b
    return tuple(map(lambda x: f'{round(x / (60 if x < 3600 else 3600), 2)} {["minutes","hours"][x > 3600]}', (C * s + T(e, b), s * T(e, b))))


def train_estimate(args, l=31500, M=0.007):
    e, b = args['fit']['epochs'], args['fit']['batch_size']
    return 1.26 + e * M * l / b
        

class BarGraph:
    def __init__(self, window, grid_opts, width, height, colors):
        self.w, self.h = width, height - 50
        self.colors = colors
        
        self.canvas = Canvas(window, bg='black', width=width, height=height, highlightthickness=0)
        self.canvas.grid(**grid_opts)
        
        self.spacing = width / 10
        self.padding = self.spacing / 5
        self.ceiling = 50
        self.floor = height - 50
        
        self._add_labels()
        
    def _add_labels(self):
        x = self.spacing / 2
        for digit in range(10):
            self.canvas.create_text(x, self.floor + 30, text=str(digit), fill='white', tags=('labels',))
            x += self.spacing
            self.draw_bar(digit, 0.1)
            
    def _get_bar_coords(self, digit):
        t = self.spacing * digit
        return t + self.padding, t + 4 * self.padding
    
    def clear(self): self.canvas.delete('bars')
    
    def redraw(self, sizes):
        self.clear()
        [self.draw_bar(d, sizes[d]) for d in range(10)]
        
    def draw_bar(self, digit, height):
        s, e = self._get_bar_coords(digit)
        self.canvas.create_rectangle(s, self.h * (1 - height), e, self.floor, fill=self.colors[digit], tags=('bars',), outline='')


class DataDisplay:
    def __init__(self, X_n, y, pred, size=(850, 800), image_size=(28, 28)):
        self.pred = pred
        self.X_n, self.y = X_n, y
        self.X = X_n[0]
        self.size = len(y)
        self.wrapper = 0
        
        w, h = size
        iw, ih = image_size
        self.x_scale, self.y_scale = (w - 50) // iw, (h - 100) // ih
        self.w, self.h = image_size
        
        self.root = Tk()
        self.root.geometry(f'{w}x{h}')
        self.root.configure(bg='black')
        
        self.root.lift()
        self.root.attributes('-topmost', True)
        self.root.after_idle(self.root.attributes, '-topmost', False)

        self.drawing = Drawing(self.root, iw * self.x_scale, ih * self.y_scale, image_size, row=1, column=1, columnspan=3, rowspan=10)
        
        self.answer_label = Label(self.root, text='answer:  ', background='black', foreground='white')
        self.answer_label.grid(row=0, column=1)

        self.guess_label = Label(self.root, text='guess:   ', background='black', foreground='white')
        self.guess_label.grid(row=0, column=3)
        
        self.root.bind('<q>', self.quit)

        self.next_wrong = Button(self.root, text='wrong', command=self.wrong, background='black', foreground='white')
        self.next_wrong.grid(row=11, column=1)
        
        self.next_random = Button(self.root, text='random', command=self.random, background='black', foreground='white')
        self.next_random.grid(row=11, column=2)
        
        self.next_correct = Button(self.root, text='correct', command=self.right, background='black', foreground='white')
        self.next_correct.grid(row=11, column=3)
        
        def c(i): Button(self.root, text=str(i), command=lambda: self.set_wrapper(i), background='black', foreground='white').grid(row=1 + i, column=0)
        [c(i) for i in range(len(X_n))]
        def c(i): Button(self.root, text=str(i), command=lambda: self.specific(i), background='black', foreground='white').grid(row=1 + i, column=4)
        [c(digit) for digit in range(10)]
        
        self.current = 0
        self.root.mainloop()
        
    def set_wrapper(self, i):
        self.wrapper = i
        self.X = self.X_n[self.wrapper]
        self.draw(self.current)
        
    def wrong(self, *_): self.draw(choice([i for i, y in enumerate(self.y) if self.pred[i] != y]))
        
    def right(self, *_): self.draw(choice([i for i, y in enumerate(self.y) if self.pred[i] == y]))
   
    def specific(self, digit):
        self.draw(choice([i for i, y in enumerate(self.y) if y == digit]))
            
    def random(self, *_): self.draw(randint(0, self.size - 1))
        
    def quit(self, *_): self.root.destroy()
    
    def draw(self, i):
        self.current = i
        
        self.answer_label['text'] = f'answer: {self.y[i][0]}'
        self.guess_label['text'] = f'guess: {self.pred[i]}'
        
        self.drawing.draw(self.X.iloc[i].values)
        
         
class Drawing:
    def __init__(self, window, w, h, image_size, **kwargs):
        self.iw, self.ih = image_size
        self.sx, self.sy = w / self.iw, h  / self.ih
        self.canvas = Canvas(window, width=w, height=h, bg='black')
        self.canvas.grid(**kwargs)
        
    def draw(self, row):
        self.canvas.delete('all')
        y = 0
        for i in range(self.ih):
            x = 0
            for j in range(self.iw):
                self.canvas.create_rectangle(x, y, x + self.sx, y + self.sy, fill=f'#{f"{int(row[i * self.ih + j]*255):02x}" * 3}')
                x += self.sx
            y += self.sy
            
            
class PredictionWindow:
    q = sqrt(2) / 2
    ms = 500
    
    @staticmethod
    def _generate_n_colors(n):
        def conv(r,g,b): return f'#{r:02x}{g:02x}{b:02x}'
        return [conv(*[randint(100,255) for _ in range(3)]) for _ in range(n)]
        
    @classmethod
    def from_Xy(cls, X, y, main_params, secondary_params, errors, *args, **kwargs):
        m = Model.from_dict(X, y, main_params)

        m.get_secondaries(errors, secondary_params)
        if errors and (lambda a, b: a > b)(*m.history):
            m.toggle_secondaries()
        
        return cls(m, *args, **kwargs)
    
    def __init__(self, model, decays, window_dim=(28 * 25 + 600, 28 * 25 + 100), image_dim=(28,28)):
        w, h = window_dim
        iw, ih = image_dim
        self.x_scale, self.y_scale = (w - 600) // iw, (h - 100) // ih

        self.model = model
        self.decays = decays
        self.decay = decays[0]
        
        self.root = Tk()
        self.root.geometry(f'{w}x{h}+100+0')
        self.root.configure(bg='black')
        
        self.root.lift()
        self.root.attributes('-topmost', True)
        self.root.after_idle(self.root.attributes, '-topmost', False)

        self.canvas = Canvas(self.root, width=iw * self.x_scale, height=ih * self.y_scale, bg='black')
        self.canvas.grid(row=0, column=1, rowspan=10, columnspan=3)

        self.digit_labels = [Label(self.root, text=f'{i}: {0:>10.2%}', background='black', foreground='white') for i in range(10)]
        for i, l in enumerate(self.digit_labels): l.grid(row=i, column=0)
        
        self.answer_label = Label(self.root, text='guess:  ', background='black', foreground='white')
        self.answer_label.grid(row=11, column=2)

        self.root.bind('<BackSpace>', self.clear)
        self.root.bind('<Return>', self.predict)
        self.root.bind('<q>', self.quit)

        self.array = [[0 for _ in range(iw)] for _ in range(ih)]
        self.drawings = [[None] * iw for _ in range(ih)]
        
        colors = ['red', 'green', 'orange', 'blue', 'purple', 'yellow'] + self._generate_n_colors(4)
        self.bars = BarGraph(self.root, {'row':2, 'column':4, 'rowspan':7, 'columnspan':len(decays)}, 500, h - 100, colors)
        
        self.output_display = Drawing(self.root, 250, 250, image_dim, row=1, rowspan=3, column=4, columnspan=len(decays))
        
        def c(i): Button(self.root, text=f'decay #{i}', command=lambda: self.change_decay(i), background='black', foreground='white').grid(row=0, column=i + 4)
        [c(i) for i in range(len(decays))]
        
        self.mouse_down = False
        self.mouse_type = 1
        self.canvas.bind('<ButtonPress-1>', lambda e: (self.toggle_down(1), self.draw(e.x, e.y)))
        self.canvas.bind('<ButtonRelease-1>', lambda _: self.toggle_down(1))

        self.canvas.bind('<ButtonPress-3>', lambda e: (self.toggle_down(3), self.erase(e.x, e.y)))
        self.canvas.bind('<ButtonRelease-3>', lambda _: self.toggle_down(3))

        self.canvas.bind('<Motion>', lambda e: self.drag(e.x, e.y))
        
        self.paused = False
        self.root.bind('<space>', self.toggle_pause)
        
        self.predictions = []
        self.root.after(self.ms, self.loop)
        self.root.mainloop()
        
    def change_decay(self, i):
        self.decay = self.decays[i]
        self.predict()
        
    def toggle_paused(self, *_): self.paused = not self.paused

    def toggle_down(self, type):
        self.mouse_down = not self.mouse_down
        self.mouse_type = type

    def drag(self, x, y):
        if not self.mouse_down: return

        if self.mouse_type == 1: self.draw(x, y)
        elif self.mouse_type == 3: self.erase(x, y)

    def draw(self, x, y):  
        i, j = y // self.x_scale, x // self.y_scale
        self.array[i][j] = 1

        x0, y0 = x - x % self.x_scale, y - y % self.y_scale
        self.canvas.create_rectangle(x0, y0, x0 + self.x_scale, y0 + self.y_scale,
                                     fill='white', tags=('drawings', f'{i},{j}'), outline='')

    def erase(self, x, y):
        i, j = y // self.x_scale, x // self.y_scale
        if self.array[i][j]:
            self.array[i][j] = 0
            self.canvas.delete(f'{i},{j}')

    def clear(self, *_): 
        self.array = [[0 for _ in range(len(r))] for r in self.array]
        self.drawings = [[None] * len(r) for r in self.drawings]
        self.canvas.delete('drawings')
    
    def predict(self, *_): 
        if not any(any(a) for a in self.array):
            return
        
        a = list(flatten.from_iterable(self.array)) #self.decay(minimum_dist(self.array))
        self.output_display.draw(a)
        
        p = self.model.predict(np.array([a]), verbose=0)[0]
        p /= sum(p)
        self.bars.redraw(p)
        for digit, result in enumerate(p):
            self.digit_labels[digit]['text'] = f'{digit}: {result:>10.2%}'
            
        self.answer_label['text'] = f'guess: {p.argmax()}'
        self.predictions.append((a, p.argmax()))
        return p
                       
    def quit(self, *_): 
        self.root.destroy()
      
    def display_output(self, y):
        a, d = list(zip(*self.predictions))
        DataDisplay(pd.DataFrame(a), y, d)
        
    def toggle_pause(self, *_): self.paused = not self.paused
    
    def loop(self):
        if self.paused:
            return self.root.after(self.ms, self.loop)
        
        self.predict()
        return self.root.after(self.ms, self.loop)


class Model:
    def __init__(self, X, y, *args, load=False, **kwargs):
        if 'imports' in kwargs:
            globals().update({k.__name__: k for k in kwargs['imports']})
            del kwargs['imports']
        
        if load:
            if 'model' not in kwargs:
                raise Exception('no "model" keyword argument passed although load=True')
            if 'dictionary' not in kwargs:
                raise Exception('no "dictionary" keyword argument passed although load=True')
                
            self.model = kwargs['model']
            self.__dict__.update(kwargs['dictionary'])
            
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y)
            self.score = self._score()
        else:
            self.create_new(X, y, *args, **kwargs)
            
    @classmethod
    def from_dict(cls, X, y, params): return cls(X, y, **params['model']).fit(**params['fit'])
        
    def create_new(self, X, y, output_size, hidden_layers=2, activation='sigmoid', optimizer='adam',
                   hidden_size=None, loss_fn="mean_squared_error", metric='accuracy', hidden_inc=None, binary=False):
        self.predict = self._predict
        self.secondaries = None
        self._using_secondaries = False
        self.binary = binary
        self.history = []
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y)
            
        self.model = Sequential()
        
        size = len(X.iloc[0])
        hidden_size = int(hidden_size * size) if hidden_size else 2 * size // 3 + output_size
        hidden_inc = [1] * hidden_layers if hidden_inc is None else hidden_inc
        
        self.model.add(Dense(int(hidden_size * hidden_inc[0]), input_shape=(size,), activation=activation))
        for inc in hidden_inc[1:]: self.model.add(Dense(int(hidden_size * inc), activation=activation))
            
        self.model.add(Dense(output_size, activation=activation))
        
        self.model.compile(loss=loss_fn, optimizer=optimizer, metrics=metric)
        
    def fit(self, epochs=30, batch_size=105, verbose=0):
        self.model.fit(self.X_train, self.y_train, validation_data=(self.X_test, self.y_test), epochs=epochs, 
                       batch_size=batch_size, verbose=verbose, workers=10, use_multiprocessing=True)
        self.score = self._accuracy(self._predict(self.X_test, verbose=verbose, workers=10, use_multiprocessing=True), self.y_test)
        self.history.append(self.score)
        return self

    
    def _get_threshold(self, a, b): return (lambda x: sum(x) / len(x))([abs(p[a] - p[b]) for p, e in zip(self.predict(self.X_train, verbose=0), self.y_train) if not (p.argmax() == e or e not in (a, b))])

    def _get_specific_data(self, a, b): return self.X_train.iloc[[i for i in range(len(self.y_train)) if self.y_train[i] in (a, b)]], np.array([int(temp == b) for temp in self.y_train if temp in (a, b)]).reshape((-1, 1))

    def _train_specific_binary(self, a, b, model, fit): return Model(*self._get_specific_data(a, b), **model, binary=True).fit(**fit)

    def get_secondaries(self, targets, params):
        if not targets:
            return
        
        self.secondaries = {(a, b): (self._train_specific_binary(a, b, **params), self._get_threshold(a, b)) for a, b in targets}
        
        self._using_secondaries = True
        self.predict, self.score = self._predict_with_secondaries, self._accuracy(self._predict_with_secondaries(self.X_test, verbose=0), self.y_test)
        self.history.append(self.score)
        
    def toggle_secondaries(self):
        self._using_secondaries = not self._using_secondaries
        if self.secondaries:
            self.score = self.history[self._using_secondaries]
            
    def _accuracy(self, a, b): return (lambda x: sum(x) / len(b))((lambda x: x.argmax(), lambda x: round(x[0], 0))[self.binary](n) == m for n, m in zip(a, b))[0]
    
    def display_row(i): Image.fromarray(np.reshape(np.array(self.X.iloc[i].values), (28, 28)).astype('uint8')).show()
        
    def save(self):
        open('attributes.txt', 'w').write('\n'.join(f'{k},{v},{type(v).__name__}' for k, v in self.__dict__.items() if k not in ('self', 'model', 'X_train', 'X_test', 'y_train', 'y_test')))
        self.model.save('model.h5', overwrite=True)
        
    @classmethod
    def load(cls, X, y, model_file='model.h5', attr_file='attributes.txt'): 
        return cls(X, y, model=load_model(model_file), dictionary={k: globals()['__builtins__'].__dict__[t](v) for k, v, t in [a.split(',') for a in open(attr_file).read().split('\n')]}, load=True)

    @property
    def summary(self): return self.model.summary()

    def __call__(self, *args, **kwargs): return self.predict(*args, **kwargs)

    def _predict(self, data, *args, **kwargs): return self.model.predict(data, *args, **kwargs)

    def _predict_with_secondaries(self, data, *args, **kwargs):
        predictions = self._predict(data, *args, **kwargs)
        if not self._using_secondaries:
            return predictions
        
        get = (lambda x, i: x.iloc[i]) if type(data) == pd.DataFrame else (lambda x, i: x[i])
        for (a, b), (model, threshold) in self.secondaries.items():
            p = {i: get(data, i) for i, x in enumerate(predictions) if x.argmax() in (a, b) and abs(x[a] - x[b]) > 0}  # <= threshold
            if not p:
                continue
            for i, x in zip(p, model.predict(np.array(list(p.values())), *args, **kwargs)):
                predictions[i] = [0 for _ in predictions[i]]
                predictions[i][a], predictions[i][b] = 1 - x[0], x[0]
        return predictions
        

In [8]:
def minimum_dist(data):    
    if isinstance(data, pd.Series):
        data = np.array(data.values).reshape((28, 28))
    else:
        data = np.array(data)

    x, y = np.where(data != 0)
    x_, y_ = np.where(np.ones(data.shape))
    return np.sqrt(np.subtract.outer(x, x_) ** 2 + np.subtract.outer(y, y_) ** 2).min(0)

k_n = [
    0.4,
    2.1 / pi,
    1 / np.arccosh(15),
    2,
    0.067
]

decays = [
    lambda x: x,
    lambda x, k=k_n[0], c=15: (2 / (1 + np.exp(x * k))),
    lambda x, k=k_n[1], c=13: (1 - k * np.arctan(x)),
    lambda x, k=k_n[2], c=15: (1 - k * np.arccosh(x + 1)),
    lambda x, k=k_n[3], c=4:  (2 / (1 + np.exp(x * k))),
    lambda x, k=k_n[4], c=15: ((x * k - 1) ** 2)
]


basic = train.apply(minimum_dist, axis=1, result_type='expand')
distances = [pd.DataFrame(decay(basic)) for decay in map(np.vectorize, decays)]

In [18]:
main = PredictionWindow.from_Xy(distances[1], y, main_params, secondary_params, errors, decays=decays).model
predictions = [x.argmax() for x in main.predict(distances[1], verbose=0)]

329/329 [==============================] - 2s 5ms/step


In [59]:
scores = []
datasets = [train, *distances]
for i, t in enumerate(datasets):
    main = Model.from_dict(t, y, main_params)
    main.get_secondaries([(3, 8)], secondary_params)

    print(i, main.history)
    scores.append(main.history)

scores

0 [0.9767619047619047, 0.9753333333333334]
1 [0.9682857142857143, 0.9681904761904762]
2 [0.9732380952380952, 0.9729523809523809]
3 [0.9704761904761905, 0.9694285714285714]
4 [0.9743809523809523, 0.9723809523809523]
5 [0.9552380952380952, 0.9553333333333334]


[[0.9767619047619047, 0.9753333333333334],
 [0.9682857142857143, 0.9681904761904762],
 [0.9732380952380952, 0.9729523809523809],
 [0.9704761904761905, 0.9694285714285714],
 [0.9743809523809523, 0.9723809523809523],
 [0.9552380952380952, 0.9553333333333334]]

In [64]:
s[s.argmax()].argmax()

0

In [10]:
predictions = [x.argmax() for x in main.predict(train_scaled, verbose=0)]

In [19]:
_ = PredictionWindow(main, decays)

In [ ]:
_ = DataDisplay(distances, y, predictions)

In [54]:
save_output(main, test)

175/175 [==============================] - 1s 5ms/step
Successfully submitted to Digit Recognizer



  0%|          | 0.00/235k [00:00<?, ?B/s]
 37%|###7      | 88.0k/235k [00:00<00:00, 896kB/s]
100%|##########| 235k/235k [00:01<00:00, 218kB/s] 
